In [ ]:
!pip install numin

In [ ]:
import pandas as pd
import time
import os
from numin import NuminAPI

# Constants
API_KEY = "5d03ab5f-786a-790c-1f94-2f8fcd487557"  # Your API key
WAIT_INTERVAL = 300  # Time (in seconds) to wait before checking the round number again
NUM_ROUNDS = 20  # Number of rounds to test

# Create temp directory if it doesn't exist
os.makedirs("API_submission_temp", exist_ok=True)

# Initialize NuminAPI instance
api_client = NuminAPI(api_key=API_KEY)

print("NuminAPI instance initialized successfully!")

# 2. Download training data
file_path = "training_data.zip"
print("Downloading training data...")
data = api_client.get_data(data_type="training")  # BytesIO
# if isinstance(data, dict) and "error" in data:
#     print(f"Failed to download training data: {data['error']}")
#     exit(1)

# with open(file_path, 'wb') as f:
#     f.write(data.getbuffer())

# def process_round_data(df, current_round):
#     predictions = []
#     for ticker in df['id'].unique():
#         ticker_data = df[df['id'] == ticker].copy()
#         # Get predictions for this ticker
#         target_5, target_10 = make_predictions(ticker_data)
#         predictions.append({
#             "id": ticker,
#             "predictions": target_10,  # Using target_10 as our prediction
#             "round_no": int(current_round)
#         })
#     return pd.DataFrame(predictions)

# Main testing loop
previous_round = None
rounds_completed = 0

while rounds_completed < NUM_ROUNDS:
    try:
        # 4. Get current round number
        current_round = api_client.get_current_round()
        # if isinstance(current_round, dict) and "error" in current_round:
        #     print(f"Error getting round number: {current_round['error']}")
        #     time.sleep(WAIT_INTERVAL)
        #     continue

        # print(f"Current Round: {current_round}")

        # Check if round has changed
        if current_round != previous_round:
            # 3. Download round data
            print("Downloading round data...")
            round_data = api_client.get_data(data_type="round")
            # if isinstance(round_data, dict) and "error" in round_data:
            #     print(f"Failed to download round data: {round_data['error']}")
            #     time.sleep(WAIT_INTERVAL)
            #     continue

            # 5. Process data and create predictions
            print("Generating predictions...")
            predictions_df = make_predictions(round_data, current_round) # this fucntion needs to be finalised

            # Save predictions to temporary CSV
            temp_csv_path = "API_submission_temp/predictions.csv"
            predictions_df.to_csv(temp_csv_path, index=False)

            # 6. Submit predictions - needs to be modified
            print("Submitting predictions...")
            submission_response = api_client.submit_predictions(temp_csv_path)
            if isinstance(submission_response, dict) and "error" in submission_response:
                print(f"Failed to submit predictions: {submission_response['error']}")
            else:
                print("Predictions submitted successfully!")
                rounds_completed += 1
                previous_round = current_round

        else:
            print("Waiting for next round...")

        time.sleep(WAIT_INTERVAL)

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        time.sleep(WAIT_INTERVAL)

# print("\nTesting completed.")

NuminAPI instance initialized successfully!
Training data downloaded and saved to training_data.zip
Current Round: 148
Generating predictions...
Submitting predictions...
Failed to submit predictions: Failed to submit predictions: Invalid API key
Current Round: 148
Generating predictions...
Submitting predictions...
Failed to submit predictions: Failed to submit predictions: Invalid API key
Current Round: 148
Generating predictions...
Submitting predictions...
Failed to submit predictions: Failed to submit predictions: Invalid API key
Current Round: 148
Generating predictions...
Submitting predictions...
Failed to submit predictions: Failed to submit predictions: Invalid API key
Current Round: 148
An error occurred: Response ended prematurely
Current Round: 148
Generating predictions...
Submitting predictions...
Failed to submit predictions: Failed to submit predictions: Invalid API key
Current Round: 148
An error occurred: Response ended prematurely
Current Round: 148
Generating predi

KeyboardInterrupt: 

In [ ]:
def make_predictions_rsi(df, span=14):
    # Get the latest row of data
    current = df.iloc[-1]

    # Initialize prediction scores
    score_5 = 0
    score_10 = 0

    # RSI Analysis
    rsi = current['RSI_14']

    if rsi > 70:  # Overbought
        score_5 -= 0.3
        score_10 -= 0.3
    elif rsi < 30:  # Oversold
        score_5 += 0.3
        score_10 += 0.3

    # Volume Analysis
    volume = current['Volume_n_val']
    vol_sma = current['VOL_SMA_20']

    if volume > vol_sma:
        score_5 *= 1.2
        score_10 *= 1.2

    # Normalize scores to be between 0 and 4
    score_5 = int(2 * (score_5 + 1))
    score_10 = int(2 * (score_10 + 1))

    return score_5, score_10

In [ ]:
def make_predictions_sma_ema(df, short_span=10, long_span=50):
    # Get the latest row of data
    current = df.iloc[-1]

    # Initialize prediction scores
    score_5 = 0
    score_10 = 0

    # SMA and EMA Crossover
    sma_short = current[f'SMA_{short_span}']
    sma_long = current[f'SMA_{long_span}']
    ema_short = current[f'EMA_{short_span}']
    ema_long = current[f'EMA_{long_span}']

    if sma_short > sma_long and ema_short > ema_long:  # Bullish signal
        score_5 += 0.3
        score_10 += 0.3
    elif sma_short < sma_long and ema_short < ema_long:  # Bearish signal
        score_5 -= 0.3
        score_10 -= 0.3

    # Price Momentum
    momentum = current['MOM_30']
    if momentum > 0:
        score_5 += 0.2
        score_10 += 0.2
    else:
        score_5 -= 0.2
        score_10 -= 0.2

    # Normalize scores
    score_5 = int(2 * (score_5 + 1))
    score_10 = int(2 * (score_10 + 1))

    return score_5, score_10

In [ ]:
def make_predictions_stochastic(df):
    # Get the latest row of data
    current = df.iloc[-1]

    # Initialize prediction scores
    score_5 = 0
    score_10 = 0

    # Stochastic Oscillator
    stochastic_k = current['Stoch_K']
    stochastic_d = current['Stoch_D']

    if stochastic_k > 80 and stochastic_d > 80:  # Overbought
        score_5 -= 0.3
        score_10 -= 0.3
    elif stochastic_k < 20 and stochastic_d < 20:  # Oversold
        score_5 += 0.3
        score_10 += 0.3

    # Normalize scores
    score_5 = int(2 * (score_5 + 1))
    score_10 = int(2 * (score_10 + 1))

    return score_5, score_10

In [ ]:
def make_predictions_donchian(df, channel_period=20):
    # Get the latest row of data
    current = df.iloc[-1]

    # Initialize prediction scores
    score_5 = 0
    score_10 = 0

    # Donchian Channel
    upper_band = current[f'DC_Upper_{channel_period}']
    lower_band = current[f'DC_Lower_{channel_period}']
    close = current['Close_n']

    if close > upper_band:  # Breakout above upper band
        score_5 += 0.4
        score_10 += 0.4
    elif close < lower_band:  # Breakout below lower band
        score_5 -= 0.4
        score_10 -= 0.4

    # Normalize scores
    score_5 = int(2 * (score_5 + 1))
    score_10 = int(2 * (score_10 + 1))

    return score_5, score_10

In [ ]:
def make_predictions(df, span=15): # this is the function that needs to be updated every time we get a new stratergy
    # Get latest row of data
    current = df.iloc[-1]

    # Initialize prediction scores
    score_5 = 0
    score_10 = 0

    # 1. MACD Analysis (pre-calculated in df)
    macd = current['MACD_12_26_9']
    macd_hist = current['MACDh_12_26_9']
    macd_signal = current['MACDs_12_26_9']

    if macd > 0 and macd_hist > 0:
        score_5 += 0.3
        score_10 += 0.2
    elif macd < 0 and macd_hist < 0:
        score_5 -= 0.3
        score_10 -= 0.2

    # 2. Bollinger Bands Analysis
    bb_upper = current['BBU_5_2.0']
    bb_middle = current['BBM_5_2.0']
    bb_lower = current['BBL_5_2.0']
    bb_percent = current['BBP_5_2.0']

    if bb_percent > 0.8:  # Price near upper band
        score_5 -= 0.2
        score_10 -= 0.3
    elif bb_percent < 0.2:  # Price near lower band
        score_5 += 0.2
        score_10 += 0.3

    # 3. Volume Analysis
    volume = current['Volume_n_val']
    vol_sma = current['VOL_SMA_20']

    if volume > vol_sma:  # Higher than average volume
        # Amplify existing signals
        score_5 *= 1.2
        score_10 *= 1.2

    # 4. Momentum Analysis
    momentum = current['MOM_30']
    if momentum > 0:
        score_5 += 0.2
        score_10 += 0.3
    else:
        score_5 -= 0.2
        score_10 -= 0.3

    # 5. VWAP Analysis
    vwap = current['VWAP_D']
    close = current['Close_n']

    if close > vwap:
        score_5 += 0.2
        score_10 += 0.2
    else:
        score_5 -= 0.2
        score_10 -= 0.2

    # Normalize scores to be between 0 and 4
    score_5 = int(2 * (score_5 + 1))  # Convert from [-1,1] to [0,4]
    score_10 = int(2 * (score_10 + 1))  # Convert from [-1,1] to [0,4]

    return score_5, score_10

In [ ]:
def make_predictions_ichimoku(df):
    # Get the latest row of data
    current = df.iloc[-1]

    # Initialize prediction scores
    score_5 = 0
    score_10 = 0

    # Ichimoku Cloud
    tenkan = current['Tenkan']
    kijun = current['Kijun']
    senkou_a = current['Senkou_A']
    senkou_b = current['Senkou_B']
    close = current['Close_n']

    if close > senkou_a and close > senkou_b:  # Bullish above cloud
        score_5 += 0.4
        score_10 += 0.3
    elif close < senkou_a and close < senkou_b:  # Bearish below cloud
        score_5 -= 0.4
        score_10 -= 0.3

    # Normalize scores
    score_5 = int(2 * (score_5 + 1))
    score_10 = int(2 * (score_10 + 1))

    return score_5, score_10

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from numin import NuminAPI

# Constants
API_KEY = "your_api_key_here"
BATCH_SIZE = 32
NUM_EPOCHS = 10
LEARNING_RATE = 0.01
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize API Client
api_client = NuminAPI(api_key=API_KEY)

# Step 1: Fetch and Load Training Data
print("Fetching training data...")
training_data = api_client.get_data(data_type="training")
with open("training_data.csv", "wb") as f:
    f.write(training_data.getbuffer())
df = pd.read_csv("training_data.csv")
df.dropna(inplace=True)

# Preprocess data
X = df.iloc[:, :-2].values  # Features
y = df.iloc[:, -1].apply(lambda val: int(2 * (val + 1))).values  # Convert labels from [-1,1] to [0,4]

# Define Dataset
class NuminDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.tensor(data, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Step 2: Define and Train the MLP Model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, x):
        return self.layers(x)

print("Training the MLP model...")
dataset = NuminDataset(X, y)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
input_size = X.shape[1]
mlp = MLP(input_size, hidden_size=100, output_size=5).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=LEARNING_RATE)

for epoch in range(NUM_EPOCHS):
    for features, labels in dataloader:
        features, labels = features.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = mlp(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Loss: {loss.item():.4f}")

# Step 3: Fetch Round Data, Predict, and Submit
print("Fetching round data and making predictions...")
round_data = api_client.get_data(data_type="round")
with open("round_data.csv", "wb") as f:
    f.write(round_data.getbuffer())
round_df = pd.read_csv("round_data.csv")
round_X = torch.tensor(round_df.iloc[:, :-2].values, dtype=torch.float32).to(DEVICE)

# Make Predictions
mlp.eval()
with torch.no_grad():
    predictions = mlp(round_X)
    predicted_classes = predictions.argmax(dim=1).cpu().numpy()

# Prepare and Submit Predictions
submission_df = pd.DataFrame({
    "id": round_df["id"],
    "predictions": predicted_classes
})
submission_df.to_csv("predictions.csv", index=False)
print("Submitting predictions...")
response = api_client.submit_predictions("predictions.csv")
if "error" in response:
    print(f"Failed to submit predictions: {response['error']}")
else:
    print("Predictions submitted successfully!")